## Homework 4, Problem 4 Classification on real data

ECE C143A/C243A, Spring Quarter 2023, Prof. J.C. Kao, TAs T. Monsoor, R. Gore, D. Singla

## Background
Neural prosthetic systems can be built based on classifying neural activity related to planning. As described in class, this is analogous to mapping patterns of neural activity to keys on a keyboard.
In this problem, we will apply the results of Problems 1 and 2 to real neural data. The neural data were recorded using a 100-electrode array in premotor cortex of a macaque monkey1. The dataset can be found on BruinLearn as `ps4_realdata.mat`.

The following describes the data format. The `.mat` file is loaded into Python as a dictionary with two keys: `train_trial` contains the training data and `test_trial` contains the test data. Each of these contains spike trains recorded simultaneously from 97 neurons while the monkey reached 91 times along each of 8 different reaching angles.

The spike train recorded from the $i_{th}$ neuron on the $n_{th}$ trial of the $k_{th}$ reaching angle is accessed as 

`data['train_trial'][n,k][1][i,:]`

where n = 0,...,90, k = 0,...,7, and i = 0, . . . , 96.  The [1] in between [n,k] and [i,:] does not mean anything for this assignment and is simply an "artifact" of how the data is structured. A spike train is represented as a sequence of zeros and ones, where time is discretized in 1 ms steps. A zero indicates that the neuron did not spike in the 1 ms bin, whereas a one indicates that the neuron spiked once in the 1 ms bin. The structure test trial has the same format as train trial.

Each spike train is 700 ms long (and thus represented by an array of length 700).  This comprises a 200ms baseline period (before the reach target turned on), a 500ms planning period (after the reach target turned on).  Because it takes time for information about the reach target to arrive in premotor cortex (due to the time required for action potentials to propagate and for visual processing), we will ignore the first 150ms of the planning period.  *** FOR THIS PROBLEM, we will take spike counts for each neuron within a single 200ms bin starting 150ms after the reach target turns on. ***

In other words, to calculate firing rates, you will calculate it over the 200ms window: 

`data['train_trial'][n,k][1][i,350:550]`

In [182]:
import numpy as np
import numpy.matlib as npm
import matplotlib.pyplot as plt
import scipy.special
import scipy.io as sio
import math

data = sio.loadmat('ps4_realdata.mat') # load the .mat file.
NumTrainData = data['train_trial'].shape[0]
NumClass = data['train_trial'].shape[1]
NumTestData = data['test_trial'].shape[0]
print(NumTrainData)
print(NumClass)
# Reloading any code written in external .py files.
%load_ext autoreload
%autoreload 2

91
8
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### (a) (8 points) 
Fit the ML parameters of model i) to the training data (91 × 8 observations of a length 97 array of neuron firing rates). 

To calculate the firing rates, use a single 200ms bin starting from 150ms after the target turns on.  This corresponds to using `data['train_trial'][n,k][1][i, 350:550]` to calculate all firing rates.  This corresponds to a 200ms window that turns on 150ms after the reach turns on.

Then, use these parameters to classify the test data (91 × 8 data points) according to the decision rule (1). What is the percent of test data points correctly classified?

In [183]:
##4a

# Calculate the firing rates.

trainDataArr =  np.zeros((NumClass,NumTrainData,97)) # contains the firing rates for all neurons on all 8 x 91 trials in the training set
testDataArr =  np.zeros((NumClass,NumTestData,97)) # for the testing set.

for classIX in range(NumClass): # 8
    for trainDataIX in range(NumTrainData):   # 91
        trainDataArr[classIX,trainDataIX,:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData):  # 91      
        testDataArr[classIX,testDataIX,:]=np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)
#====================================================#
# YOUR CODE HERE:
#   Fit the ML parameters of model i) to training data
#====================================================#

# trainDataArr[0] = class 1
# trainDataArr[class][91 trials][97 neurons]

pis = []
tot_n = 8*91
for classIX in range(8):
    curr_n = len(trainDataArr[classIX])
    pis += [curr_n/tot_n]
# pis = 8 len array of each class's pi

modParam1 = {}
modParam1['pi'] = pis
modParam1['cov'] = np.zeros((97,97)) 
modParam1['mean'] = np.zeros((NumClass,97))   

for classIX in range(NumClass):
    for i in range(97): modParam1['mean'][classIX][i] = np.mean(trainDataArr[classIX][:,i]) 
for classIX in range(NumClass): modParam1['cov'] += (trainDataArr[classIX].shape[0])/tot_n*(np.cov(trainDataArr[classIX].T, bias=True))
    
# COVARIANCE STEPS:
#     subtract class specific means for each class from each neuron
#     feed in [neuron1, neuron2] where data has had means subtracted
#====================================================#
# END YOUR CODE
#====================================================# 

In [190]:
#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#
dic = np.zeros((8,91))
prob = np.zeros(8)
acc = 0

for tria in range(91):
    for i in range(8):
        y = testDataArr[i][tria][:]
        for classIX in range(8):
            meanIX = modParam1['mean'][classIX][:]
            covIX = modParam1['cov']
            prob[classIX] = -(0.5)*np.matmul((y - meanIX).T, np.matmul(np.linalg.inv(covIX),y - meanIX))
        dic[i][tria] = np.argmax(prob)
        if(dic[i][tria] == i):
            acc += 1
            
print('Accuracy: ', acc/(8*91))
#====================================================#
# END YOUR CODE
#====================================================# 

Accuracy:  0.9601648351648352


#### Question:
What is the percent of test data points correctly classified?

#### Your answer: 
96 percent!

### (b) (6 points) 
Repeat part (a) for model ii). You `should encounter a Python error` when classifying the test data. What is this error? Why did the Python error occur? What would we need to do to correct this error?

To be concrete, the output of this cell should be a `Python error` and that's all fine.  But we want you to understand what the error is so we can fix it later.


In [180]:
##4b

#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters of model ii) to training data
#====================================================#
modParam2 = {} 

# trainDataArr[0] = class 1
# trainDataArr[class][91 trials][97 neurons]

pis = []
tot_n = 8*91
for classIX in range(8):
    curr_n = len(trainDataArr[classIX])
    pis += [curr_n/tot_n]
# pis = 8 len array of each class's pi

modParam2['pi'] = pis
modParam2['cov'] = np.zeros((97,97)) 
modParam2['mean'] = np.zeros((NumClass,97))   

for classIX in range(NumClass):
    for neur in range(97):
        modParam2['mean'][classIX][neur] = np.mean(trainDataArr[classIX][:,i])

modParam2['cov'] = np.zeros((NumClass,97,97))
for classIX in range(NumClass):
    modParam2['cov'][classIX] = np.cov(trainDataArr[classIX].T, bias=True)

#====================================================#
# END YOUR CODE
#====================================================# 


In [191]:
#====================================================#
# YOUR CODE HERE:
#   Classify the test data and print the accuracy
#====================================================#
dic = np.zeros((8,91))
prob = np.zeros(8)
acc = 0

for trial in range(91):
    for i in range(8):
        y = testDataArr[i][trial][:]
        for classIX in range(8):
            meanIX = modParam2['mean'][classIX][:]
            covIX = modParam2['cov']
            prob[classIX] = -(0.5)*np.matmul((y - meanIX).T, np.matmul(np.linalg.inv(covIX),y - meanIX))
        dic[i][trial] = np.argmax(prob)
        if(dic[i][trial] == i):
            acc += 1
            
print('Accuracy: ', acc/(8*91))
#====================================================#
# END YOUR CODE
#====================================================# 

LinAlgError: Singular matrix

#### Question:
Why did the python error occur? What would we need to do to correct this error?

#### Your answer:  
Because this model has class specific covariances, the matrix is not invertible. Therefore, np.linalg.inv(covIX) runs into an error.
To overcome this, we may add random noise to the covariance matrix, or remove offending neurons.

### (c) (8 points) 
Correct the problem from part (b) by detecting and then removing offending neurons that cause the error. Now, what is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

In [192]:
##4c
neuronsToRemove = []
#====================================================#
# YOUR CODE HERE:
#   Detect and then remove the offending neurons, so that 
#   you no longer run into the bug in part (b).
#====================================================#
trainArr = np.zeros((8,91,97))
testArr = np.zeros((8,91,97))
for classIX in range(8):
    for trainDataIX in range(NumTrainData): trainArr[classIX][trainDataIX][:] = np.sum(data['train_trial'][trainDataIX,classIX][1][:,350:550],1)
    for testDataIX in range(NumTestData): testArr[classIX][testDataIX][:] = np.sum(data['test_trial'][testDataIX,classIX][1][:,350:550],1)

for classIX in range(8):
    for neuron in range(97):
        fr = trainArr[classIX][:,neuron]
        if np.sum(fr) == 0:
            neuronsToRemove.append(neuron)
            
trainArr = np.delete(trainDataArr,list(set(neuronsToRemove)),2) 
testArr = np.delete(testDataArr,list(set(neuronsToRemove)),2)
#====================================================#
# END YOUR CODE
#====================================================# 
##
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
modParam2 = {} 

# trainDataArr[0] = class 1
# trainDataArr[class][91 trials][97 neurons]

pis = []
tot_n = 8*91
for classIX in range(8):
    curr_n = len(trainArr[classIX])
    pis += [curr_n/tot_n]
# pis = 8 len array of each class's pi

modParam2['pi'] = pis
modParam2['cov'] = np.zeros((87,87)) 
modParam2['mean'] = np.zeros((NumClass,87))   

for classIX in range(NumClass):
    for i in range(87):
        modParam2['mean'][classIX][i] = np.mean(trainArr[classIX][:,i])

modParam2['cov'] = np.zeros((NumClass,87,87))
for classIX in range(NumClass):
    modParam2['cov'][classIX] = np.cov(trainArr[classIX].T, bias=True)
    
dic = np.zeros((8,91))
prob = np.zeros(8)
acc2 = 0

for trial in range(91):
    for i in range(8): 
        y = testArr[i][trial][:]
        for classIX in range(8):
            meanIX = modParam2['mean'][classIX,:]
            covIX = modParam2['cov'][classIX]
            prob[classIX] = np.log(modParam2['pi'][classIX])-(1/2)*np.matmul((y-meanIX).T, np.matmul(np.linalg.inv(covIX),y-meanIX))-(1/2)*np.log(np.linalg.det(covIX))
        
        dic[i][trial] = np.argmax(prob)
       
        if(dic[i][trial] == i):
            acc2 += 1
            
print('Accuracy: ', acc2/(8*91))
#====================================================#
# END YOUR CODE
#====================================================#

Accuracy:  0.4409340659340659


#### Question:
What is the percent of test data points correctly classified? Is it higher or lower than your answer to part (a)? Why might this be?

#### Your answer:  44.1%, which is lower. We've pruned our data and lost out on some information, likely contributing to the decrease in performance. Our first model performed better.


### (d) (8 points) 
Now we classify using a naive Bayes model. Repeat part (a) for model iii). Keep the convention in part (c), where offending neurons were removed from the analysis.

In [168]:
##4d
#====================================================#
# YOUR CODE HERE:
# Fit the ML parameters,classify the test data and print the accuracy
#====================================================#
modParam3 = {}
pis = []
tot_n = 8*91
for classIX in range(8):
    curr_n = len(trainArr[classIX])
    pis += [curr_n/tot_n]
# pis = 8 len array of each class's pi
modParam3['pi'] = pis
x, dud, y = np.shape(trainArr)
mean = np.zeros((x,y))
for classIX in range(8):
    for i in range(y): mean[classIX][i] = np.mean(trainArr[classIX][:][i])
modParam3['mean'] = mean

dic2 = np.zeros((8,91))
prob2 = np.zeros(8)
acc2 = 0
param = y
for trial in range(91):
    for i in range(8):
        ar = testArr[i][trial][:]
        for classIX in range(8):
            meanIX = modParam3['mean'][classIX][:]
            # covIX = modParam3['cov']
            prb = 0
            for el in range(param): prb += np.log(modParam3['pi'][classIX]) + (ar[el]*np.log(meanIX[el])-meanIX[el])
            prob2[classIX] = prb
        dic2[i][trial] = np.argmax(prob2)
        if(dic2[i][trial] == i):
            acc2 += 1
            
print('Accuracy: ', (arr_x*acc2/(8*91)))
#====================================================#
# END YOUR CODE
#====================================================# 

Accuracy:  0.92


#### Question:
what is the percent of test data points correctly classified? 

#### Your answer: 
92%